The first preliminary step to performing any kind of sentiment analysis on Reddit data is establishing a Reddit instance via API. This can be done with the Python Reddit API Wrapper (PRAW), see documentation: https://praw.readthedocs.io/en/latest/

In order to access the API, a Reddit account is needed, login details below.
E-mail: barbara.balcon@studbocconi.it
Username: thesis_3078976
Password: class_of_2021

I created an application at the following link: https://www.reddit.com/prefs/apps
Name: Sentiment Analysis
I selected 'script' (Script for personal use. Will only have access to the developers accounts
description)
Description: Data will be used to perform sentiment analysis
About url: blank
Redirect url: http://www.example.com/unused/redirect/url (this is empty)

After creating the app, the following is generated.
ID: L_wWPUNiMBmn1Q
Secret: bUYKyV21W3xXK4GzXsY5tsNzpG3pRw



In [ ]:
#import of the relevant libraries 
import praw
import pandas as pd
from praw.models import MoreComments
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import FreqDist
import emoji #remove emoji
import re #remove links
#import en_core_web_sm
import spacy

In [ ]:
#create a reddit connection with reddit api details
reddit=praw.Reddit(client_id='L_wWPUNiMBmn1Q', client_secret='bUYKyV21W3xXK4GzXsY5tsNzpG3pRw', user_agent='ua')

The following pulls the five hottest posts in the subreddit wallstreetbets, printing the title and ID of each

In [ ]:
subreddit=reddit.subreddit('wallstreetbets')
for submission in subreddit.hot(limit=5):
    print(submission.title)
    print('Submission ID = ', submission.id, '\n')

I am about to collect data in a SQL database, I use MySQL as a databse management system. In order to do so, after installing and setting up MySQL on my machine, we need to import a python package here and to connect to the server.

Then, we create a database for storing the reddit data (which will be stramed in a subsequent step).

In [ ]:
import mysql.connector #importing the Python package to use MySQL
from unidecode import unidecode #a package for dealing with Unicode
import datetime 
import time #packages for handling date and time data

#below I am connecting to the server I have started on my machine
mydb=mysql.connector.connect(host='MacBook-Air-di-Barbara.local', 
                             user='root',
                            passwd='class_of_2021')

mycursor=mydb.cursor()

#creation of a database for storing reddit data 
mycursor.execute('CREATE DATABASE IF NOT EXISTS reddit_data')

#creating variables we are about to create instances of when analysing sentiment
mycursor.execute("""CREATE TABLE IF NOT EXISTS reddit_data.reddit_data_sentiment
                (date_time DATETIME,
                subreddit VARCHAR(500),
                title VARCHAR(500),        
                body VARCHAR(2000),
                author VARCHAR(500),
                sentiment DECIMAL(5,4)  
                )
                """)

#pushing the data to the database (we will use this variable in the next code snippet)
sqlFormula = "INSERT INTO reddit_data.reddit_data_sentiment (date_time, subreddit, title, body, author, sentiment) VALUES (%s, %s, %s, %s, %s, %s)"


Now we are ready to actually stream comments from Reddit and perform sentiment analysis via VADER (Valence Aware Dictionary and sEntiment Reasoner). It is an open-source tool that was designed for social media specifically, it is lexicon and rule-based.

It returns a polarity score (-1, +1) for each post: a normalized, weighted composite score that acts as a metric of the overall sentiment of a given post.  

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer= SentimentIntensityAnalyzer() #just an abbreviation 

while True:
    try:
        
        subreddit = reddit.subreddit('wallstreetbets')
        for comment in subreddit.stream.comments(skip_existing=True):
                current_time = datetime.datetime.now()
                subreddit = str(comment.subreddit)
                author = str(comment.author)
                title = str(comment.link_title)
                body = str(comment.body)
                
                vs = analyzer.polarity_scores(unidecode(body))
                sentiment = vs['compound'] #we are interested in the compound score
                db = (current_time,subreddit,title,body,author,sentiment)
                mycursor.execute(sqlFormula, db)
                mydb.commit()
'''We keep an exception so that in case of error we do not hit
the API multiple times''' 
    except Exception as e:
        print(str(e))
        time.sleep(10)
